<a href="https://colab.research.google.com/github/kodeu/smartphones_prices/blob/gbm/phones_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import files
import pandas as pd
import io
from sklearn.metrics import mean_squared_error, r2_score

uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=filename, length=len(uploaded[filename])))
filename = list(uploaded.keys())[0]
data = pd.read_csv(io.BytesIO(uploaded[filename]))

# Assuming 'df' is your DataFrame with features and 'price' as the target variable
X = data.drop(columns=['Final Price'])
y = data['Final Price']

# Define the preprocessing for numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Define LightGBM parameters
params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'n_estimators': 100  # Number of boosting rounds
}

# Create a pipeline with preprocessing and model training
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(**params))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"GBM RMSE: {rmse}")
print("Coefficient of Determination (R^2):", r2_score(y_test, y_pred))

Saving smartphones.csv to smartphones (12).csv
User uploaded file "smartphones (12).csv" with length 146107 bytes
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 1452, number of used features: 39
[LightGBM] [Info] Start training from score 484.612314
(1452, 7)
(1452,)
(364, 7)
(364,)


ValueError: Found unknown categories ['Apple iPhone 13 Pro Max 512GB Grafito Libre', 'Realme 9 5G 4/64GB Blanco Libre', 'Honor 10 Lite 3/64GB Negro Libre', 'Vivo Y76 5G 8/256GB Midnight Space Libre', 'ZTE Blade A31 Lite 32GB Azul Libre', 'Ulefone Armor X5 3/32Gb Rojo Libre', 'Apple iPhone 7 32GB Dorado Rosa Libre', 'Samsung Galaxy A14 4/64GB Negro Libre + Carcasa Card Slot', 'Apple iPhone 13 512GB Blanco Estrella Libre', 'Xiaomi Mi 8 Lite 4GB/64GB Dual Sim Negro Libre', 'Motorola Edge 30 Neo 8/128GB Very Peri Libre', 'Blackview A90 4/64GB Lila Libre', 'Samsung Galaxy S23 Plus 512GB Verde Libre', 'Huawei Mate 20 Lite 4/64GB Negro Libre', 'Xiaomi Redmi Note 12 Pro 8/256GB Blanco Polar Libre', 'Huawei P30 Lite 4/128GB Negro Libre', 'Samsung Galaxy S23 Ultra 1TB Verde Libre', 'Google Pixel 4a 6/128GB Negro Libre', 'Samsung Galaxy S20 FE 6/128GB Lavanda Libre', 'POCO X5 Pro 5G 8/256GB Amarillo Libre', 'Apple iPhone XR 64 GB Azul Libre', 'Xiaomi Mi Mix 2 SE 8/128GB 4G Blanco Libre', 'Cubot K